In [1]:
import requests
import json
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import re
import datetime
from datetime import timedelta
from dateutil import relativedelta
import calendar
import getpass

## Configure your PostgreSQL database info here

In [2]:
host = "ec2-52-87-185-170.compute-1.amazonaws.com"
db_name = "pokemon_go"
username = "pokemon_go_role"
password = getpass.getpass()

········


## Create the Date Table

In [3]:
years = xrange(2016, 2025)

In [4]:
start_date = str(datetime.date(years[0], 1, 1))
end_date = str(datetime.date(years[-1], 12, 31))

In [5]:
all_dates = pd.date_range(start_date, end_date, freq='D')
all_dates

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', length=3288, freq='D')

## All the date functions

In [6]:
# Given a datetime timestamp, produce a datekey
def datetimeToDateKey(date):
    date_str = str(date)
    date_str = date_str.split(" ")[0]
    date_str = re.sub('-', '', date_str)
    return date_str

In [7]:
# Given a datetime timestamp, convert it to a string of just the date in YYYY-mm-dd format
def fullDate(date):
    date_str = str(date)
    date_str = date_str.split(" ")[0]
    return date_str

In [8]:
# Returns the weekday as a number
def weekdayNum(date):
    return date.isoweekday()

In [9]:
# Returns the weekday as a string in its full-length form
def weekdayStr(date):
    return date.strftime("%A")

In [10]:
# Returns the weekday as a string in its abbreviated form
def weekdayStrAbv(date):
    return date.strftime("%a")

In [11]:
# Returns the day of the month
def dayNumMonth(date):
    return date.day

In [12]:
# Numbers each day, constantly increasing from the first day
def dayNumOverall(date):
    day_one = all_dates[0]
    date_diff = date - day_one
    return date_diff.days + 1

In [13]:
# Returns the day of the month
def isWeekday(date):
    if date.isoweekday() in range(1,6):
        return "Weekday"
    else:
        return "Weekend"

In [14]:
# Returns the day of the month
def weekNum(date):
    return date.isocalendar()[1]

In [15]:
def weekBeginDate(date):
    dow = date.isoweekday()
    week_start = date - timedelta(days=(dow - 1))
    return week_start

In [16]:
def weekBeginDateKey(date):
    week_begin_date = weekBeginDate(date)
    date_key = datetimeToDateKey(week_begin_date)
    return date_key

In [17]:
# Numbers each day, constantly increasing from the first day
def weekNumOverall(date):
    # Find the date that the first week in the entire data set starts
    first_day = all_dates[0]
    first_week_start = weekBeginDate(first_day)
    
    # Find the date that starts the week of the current date
    curr_week_start = weekBeginDate(date)
    
    # Get the difference and find out how many weeks have passed
    date_diff = curr_week_start - first_week_start
    week_number = int(date_diff.days / 7.0 + 1.0)
    return week_number

In [18]:
# Returns the weekday as a number
def monthNum(date):
    return date.month

In [19]:
# Returns the weekday as a number
def monthNumOverall(date):
    start_date = all_dates[0]
    rel_date = relativedelta.relativedelta(date, start_date)
    month_diff = rel_date.years * 12 + rel_date.months
    return month_diff + 1

In [20]:
# Returns the month as a string in its full-length form
def monthStr(date):
    return date.strftime("%B")

In [21]:
# Returns the month as a string in its abbreviated form
def monthStrAbv(date):
    return date.strftime("%b")

In [22]:
# Returns the quarter in the year
def quarter(date):
    month = date.month
    quarter = month / 4 + 1
    return quarter

In [23]:
# Returns the year as a string
def year(date):
    return date.strftime("%Y")

In [24]:
# Returns the year and month as a concatenated string
def yearmo(date):
    year = date.strftime("%Y")
    month = date.strftime("%m")
    return year + month

In [25]:
# Returns whether or not the date is the last day of the month
def isMonthEnd(date):
    year = date.year
    month = date.month
    
    month_end = calendar.monthrange(year, month)[1]
    
    if (month_end == date.day):
        return "Month End"
    else:
        return "Not Month End"

In [26]:
# Use the date functions to make a dateframe

# Dates
date_dim = DataFrame(all_dates, columns=["full_date"])
date_dim["date_key"] = date_dim["full_date"].map(datetimeToDateKey)
date_dim = date_dim[['date_key', 'full_date']] # Reorder

# Days of Week
date_dim["day_of_week"] = date_dim["full_date"].map(weekdayNum)
date_dim["day_of_week_name"] = date_dim["full_date"].map(weekdayStr)
date_dim["day_of_week_name_abbrev"] = date_dim["full_date"].map(weekdayStrAbv)

date_dim["day_of_month"] = date_dim["full_date"].map(dayNumMonth)
date_dim["day_number_overall"] = date_dim["full_date"].map(dayNumOverall)
date_dim["day_number_overall"] = date_dim["full_date"].map(dayNumOverall)

date_dim["weekday_flag"] = date_dim["full_date"].map(isWeekday)
date_dim["week_number"] = date_dim["full_date"].map(weekNum)
date_dim["week_number_overall"] = date_dim["full_date"].map(weekNumOverall)

date_dim["week_begin_date"] = date_dim["full_date"].map(weekBeginDate)
date_dim["week_begin_date_key"] = date_dim["full_date"].map(weekBeginDateKey)

date_dim["month_number"] = date_dim["full_date"].map(monthNum)
date_dim["month_number_overall"] = date_dim["full_date"].map(monthNumOverall)
date_dim["month"] = date_dim["full_date"].map(monthStr)
date_dim["month_abbrev"] = date_dim["full_date"].map(monthStrAbv)

date_dim["quarter"] = date_dim["full_date"].map(quarter)

date_dim["year"] = date_dim["full_date"].map(year)
date_dim["year_month"] = date_dim["full_date"].map(yearmo)

date_dim["month_end_flag"] = date_dim["full_date"].map(isMonthEnd)

## Create the Time Table

In [27]:
from datetime import time
import math

In [28]:
# 1440 minutes in a day
minutes = xrange(0,1440)

In [29]:
# Given a minute number, return the 12-hour time label
def time_label_12(min_num):
    hours, minutes = divmod(min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%I:%M %p')

In [30]:
# Given a minute number, return the 24-hour time label
def time_label_24(min_num):
    hours, minutes = divmod(min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%H:%M')

In [31]:
# Given a minute number, return the 15 minute interval it occures in
def time_interval_15_min(min_num):
    return int(math.floor(min_num / 15.0))

In [32]:
# Given a minute number, return the 30 minute interval it occures in
def time_interval_30_min(min_num):
    return int(math.floor(min_num / 30.0))

In [33]:
# Given a minute number, return the 60 minute interval it occures in
def time_interval_60_min(min_num):
    return int(math.floor(min_num / 60.0))

In [34]:
# Given a minute number, return the 12-hour time label 
# with only hours (this takes up less space and is useful in some cases)
def label_hh(min_num):
    hours, minutes = divmod(min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%I %p')

In [35]:
# Given a minute number, return the 24-hour time label with just hours
def label_hh24(min_num):
    hours, minutes = divmod(min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%H')

In [36]:
# Given a minute number, return the 15 minute interval label for a 24-hour clock
def label_15_min_24(min_num):
    interval_num = time_interval_15_min(min_num)
    int_min_num = interval_num * 15
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%H:%M')

In [37]:
# Given a minute number, return the 30 minute interval label for a 24-hour clock
def label_30_min_24(min_num):
    interval_num = time_interval_30_min(min_num)
    int_min_num = interval_num * 30
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%H:%M')

In [38]:
# Given a minute number, return the 60 minute interval label for a 24-hour clock
def label_60_min_24(min_num):
    interval_num = time_interval_60_min(min_num)
    int_min_num = interval_num * 60
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%H:%M')

In [39]:
# Given a minute number, return the 15 minute interval label for a 12-hour clock
def label_15_min_12(min_num):
    interval_num = time_interval_15_min(min_num)
    int_min_num = interval_num * 15
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%I:%M %p')

In [40]:
# Given a minute number, return the 30 minute interval label for a 12-hour clock
def label_30_min_12(min_num):
    interval_num = time_interval_30_min(min_num)
    int_min_num = interval_num * 30
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%I:%M %p')

In [41]:
# Given a miute number, return the 60 minute interval label for a 12-hour clock
def label_60_min_12(min_num):
    interval_num = time_interval_60_min(min_num)
    int_min_num = interval_num * 60
    hours, minutes = divmod(int_min_num, 60)
    timestamp = time(hour=hours, minute=minutes)
    return time.strftime(timestamp, '%I:%M %p')

In [42]:
time_dim = DataFrame(Series(minutes), columns=["time_key"])
time_dim["time_label_24"] = time_dim["time_key"].map(time_label_24)
time_dim["time_label_12"] = time_dim["time_key"].map(time_label_12)

time_dim["time_interval_15min"] = time_dim["time_key"].map(time_interval_15_min)
time_dim["time_interval_30min"] = time_dim["time_key"].map(time_interval_30_min)
time_dim["time_interval_60min"] = time_dim["time_key"].map(time_interval_60_min)

time_dim["label_hh"] = time_dim["time_key"].map(label_hh)
time_dim["label_hh24"] = time_dim["time_key"].map(label_hh24)

time_dim["label_15min_24"] = time_dim["time_key"].map(label_15_min_24)
time_dim["label_30min_24"] = time_dim["time_key"].map(label_30_min_24)
time_dim["label_60min_24"] = time_dim["time_key"].map(label_60_min_24)

time_dim["label_15min_12"] = time_dim["time_key"].map(label_15_min_12)
time_dim["label_30min_12"] = time_dim["time_key"].map(label_30_min_12)
time_dim["label_60min_12"] = time_dim["time_key"].map(label_60_min_12)

## Write the data to the database

In [43]:
import psycopg2
from sqlalchemy import create_engine

In [44]:
date_dim.head()

,date_key,full_date,day_of_week,day_of_week_name,day_of_week_name_abbrev,day_of_month,day_number_overall,weekday_flag,week_number,week_number_overall,week_begin_date,week_begin_date_key,month_number,month_number_overall,month,month_abbrev,quarter,year,year_month,month_end_flag
0,20160101,2016-01-01,5,Friday,Fri,1,1,Weekday,53,1,2015-12-28,20151228,1,1,January,Jan,1,2016,201601,Not Month End
1,20160102,2016-01-02,6,Saturday,Sat,2,2,Weekend,53,1,2015-12-28,20151228,1,1,January,Jan,1,2016,201601,Not Month End
2,20160103,2016-01-03,7,Sunday,Sun,3,3,Weekend,53,1,2015-12-28,20151228,1,1,January,Jan,1,2016,201601,Not Month End
3,20160104,2016-01-04,1,Monday,Mon,4,4,Weekday,1,2,2016-01-04,20160104,1,1,January,Jan,1,2016,201601,Not Month End
4,20160105,2016-01-05,2,Tuesday,Tue,5,5,Weekday,1,2,2016-01-04,20160104,1,1,January,Jan,1,2016,201601,Not Month End


In [45]:
time_dim.head()

,time_key,time_label_24,time_label_12,time_interval_15min,time_interval_30min,time_interval_60min,label_hh,label_hh24,label_15min_24,label_30min_24,label_60min_24,label_15min_12,label_30min_12,label_60min_12
0,0,00:00,12:00 AM,0,0,0,12 AM,00,00:00,00:00,00:00,12:00 AM,12:00 AM,12:00 AM
1,1,00:01,12:01 AM,0,0,0,12 AM,00,00:00,00:00,00:00,12:00 AM,12:00 AM,12:00 AM
2,2,00:02,12:02 AM,0,0,0,12 AM,00,00:00,00:00,00:00,12:00 AM,12:00 AM,12:00 AM
3,3,00:03,12:03 AM,0,0,0,12 AM,00,00:00,00:00,00:00,12:00 AM,12:00 AM,12:00 AM
4,4,00:04,12:04 AM,0,0,0,12 AM,00,00:00,00:00,00:00,12:00 AM,12:00 AM,12:00 AM


In [46]:
## Export each to a CSV first so that we can use the COPY command. It's substantially more efficient.
date_dim.to_csv(path_or_buf="./date_dim.csv", index=False)
time_dim.to_csv(path_or_buf="./time_dim.csv", index=False)

### Create the database table the lazy way
We want to load the data from the CSV to save time, but we also want to programmatically create the table in postgres. This simply takes our dataframe, removes all the data but keeps the column names, and uses that to create the table. It also gives them appropriate data types (most of the time). Then, we fill the table with a copy command.

In [47]:
engine = create_engine('postgresql://' + username + ':' + password + '@' + host + '/' + db_name)

In [48]:
empty_date_dim = date_dim.copy()
empty_date_dim = empty_date_dim.drop(empty_date_dim.index[0:date_dim.shape[0]])
empty_date_dim.to_sql("date_dimension", engine, if_exists="replace", index=False)

In [49]:
empty_time_dim = time_dim.copy()
empty_time_dim = empty_time_dim.drop(empty_time_dim.index[0:time_dim.shape[0]])
empty_time_dim.to_sql("time_dimension", engine, if_exists="replace", index=False)

We have to create a new connection here. The above uses sqlalchemy, which then itself uses psycopg2, but we want to use psycopg2 directly.

In [50]:
connection_string = "dbname='" + db_name + "' "
connection_string += "user='" + username + "' "
connection_string += "host='" + host + "' "
connection_string += "password='" + password + "' "

In [51]:
# Set up a copy statement. The %s will be replaced later
sql_statement = """
    COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

In [52]:
def load_file(conn, table_name, primary_key, file_object):
    cursor = conn.cursor()
    cursor.copy_expert(sql=sql_statement % table_name, file=file_object)
    conn.commit()
    
    # Add add primary key, index and then vacuum 
    cursor.execute("ALTER TABLE " + table_name + " ADD  PRIMARY KEY (" + primary_key + ")")
    cursor.execute("VACUUM VERBOSE ANALYZE " + table_name)
    cursor.execute("CREATE INDEX ON " + table_name + " (" + primary_key + " ASC NULLS LAST);")
    cursor.close()

In [53]:
date_dim_file = open("./date_dim.csv")
date_dim_file

<open file './date_dim.csv', mode 'r' at 0x1148f5f60>

In [54]:
time_dim_file = open("./time_dim.csv")
time_dim_file

<open file './time_dim.csv', mode 'r' at 0x1148f5ed0>

In [55]:
conn = psycopg2.connect(connection_string)
conn.autocommit = True
try:
    load_file(conn, table_name='date_dimension', primary_key="date_key", file_object=date_dim_file)
finally:
    conn.close()

In [56]:
conn = psycopg2.connect(connection_string)
conn.autocommit = True
try:
    load_file(conn, table_name='time_dimension', primary_key="time_key", file_object=time_dim_file)
finally:
    conn.close()